# Sentinel-1 Preprocessing Workflow

In [1]:
import os
from pathlib import Path
from datetime import date

import gc
gc.enable()

# snappy imports
import snappy
from snappy import ProductIO, GPF

from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt

In [2]:
# scihub credentials
SCIHUB_UN = %env SCIHUB_UN
SCIHUB_PW = %env SCIHUB_WP

# request parameters - more TBD
REQUEST_AREA = "POLYGON ((6.2567138671875 51.57536261053028, 6.7160797119140625 51.57536261053028, 6.7160797119140625 51.78865666323309, 6.2567138671875 51.78865666323309, 6.2567138671875 51.57536261053028))"

# output directories 
TMP_OUTPUT_DIR = "./output/tmp/"
FNL_OUTPUT_DIR = "./output/"

In [3]:
Path(TMP_OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
Path(FNL_OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

## Download of Sentinel-1 Data

In [4]:
# request API
api = SentinelAPI(SCIHUB_UN, SCIHUB_PW)
products = api.query(
    area=REQUEST_AREA, 
    producttype='GRD',
    date=(date(2020, 1, 8), date(2020,1,9)),
    platformname = "Sentinel-1")

# just get the first item
first_product = products.popitem()
s1_raw_filename = first_product[1]['title'] + ".zip"

# download file (doesn't get downloaded if already exists)
downloaded_file = api.download(first_product[0], directory_path=FNL_OUTPUT_DIR)
downloaded_file

{'id': 'c001a5c2-db67-403e-9ad6-0c6e6a0d903d',
 'title': 'S1A_IW_GRDH_1SDV_20200108T171637_20200108T171702_030712_038551_49A5',
 'size': 1066808464,
 'md5': 'CBB5DE416FB75C93F6DE42541C0D5A2B',
 'date': datetime.datetime(2020, 1, 8, 17, 16, 37, 154000),
 'footprint': 'POLYGON((5.876818 51.839985,9.613271 52.246746,9.989347 50.751972,6.371759 50.347618,5.876818 51.839985))',
 'url': "https://scihub.copernicus.eu/apihub/odata/v1/Products('c001a5c2-db67-403e-9ad6-0c6e6a0d903d')/$value",
 'Online': True,
 'Creation Date': datetime.datetime(2020, 1, 9, 0, 29, 1, 991000),
 'Ingestion Date': datetime.datetime(2020, 1, 9, 0, 28, 54, 878000),
 'path': './output/S1A_IW_GRDH_1SDV_20200108T171637_20200108T171702_030712_038551_49A5.zip',
 'downloaded_bytes': 0}

In [5]:
import folium
from shapely import wkt, geometry

dataset_footprint = wkt.loads(downloaded_file['footprint'])
aoi_footprint = wkt.loads(REQUEST_AREA)

m = folium.Map(zoom_starts=3)
m.fit_bounds((
    (dataset_footprint.bounds[1], dataset_footprint.bounds[0]), 
    (dataset_footprint.bounds[3], dataset_footprint.bounds[2])))
folium.GeoJson(dataset_footprint).add_to(m)
folium.GeoJson(aoi_footprint).add_to(m)
m

## Preprocessing using snappy

In [6]:
# Some initial configurations
snappy.GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()
sentinel_image = ProductIO.readProduct(downloaded_file['path'])
HashMap = snappy.jpy.get_type('java.util.HashMap')

In [7]:
%%time
# Step: Apply Orbit Files
step1_output = os.path.join(TMP_OUTPUT_DIR, "step1_orbit_files") 

# parameters
parameters = HashMap()

# create and write product
step1_product = GPF.createProduct("Apply-Orbit-File", parameters, sentinel_image)
ProductIO.writeProduct(step1_product, step1_output, 'BEAM-DIMAP')  

CPU times: user 37.4 s, sys: 5.69 s, total: 43.1 s
Wall time: 48.7 s


In [8]:
%%time
# Step: Calibration
step2_output = os.path.join(TMP_OUTPUT_DIR, "step2_calibration")

parameters = HashMap()
parameters.put('outputSigmaBand', True)
parameters.put('outputImageScaleInDb', False)

step2_product = snappy.GPF.createProduct("Calibration", parameters, ProductIO.readProduct(step1_output + ".dim"))
ProductIO.writeProduct(step2_product, step2_output, 'BEAM-DIMAP')

CPU times: user 2min 11s, sys: 9.14 s, total: 2min 20s
Wall time: 2min 11s


In [9]:
%%time
# Step: Subsetting to area of interest
step3_output = os.path.join(TMP_OUTPUT_DIR, "step3_subset")

WKTReader = snappy.jpy.get_type('com.vividsolutions.jts.io.WKTReader')
geom = WKTReader().read(REQUEST_AREA)

parameters = HashMap()
parameters.put("geoRegion", geom)
parameters.put("outputImageScaleInDb", False)

step3_product = GPF.createProduct("Subset", parameters, ProductIO.readProduct(step2_output + ".dim"))
ProductIO.writeProduct(step3_product, step3_output, 'BEAM-DIMAP')

CPU times: user 1.82 s, sys: 554 ms, total: 2.38 s
Wall time: 2.03 s


In [10]:
%%time
# Step: Speckle Filtering
step4_output = os.path.join(TMP_OUTPUT_DIR, "step4_speckle")

parameters = HashMap()
parameters.put("filter", "Lee")
parameters.put("filterSizeX", 5)
parameters.put("filterSizeY", 5)

step4_product = GPF.createProduct("Speckle-Filter", parameters, ProductIO.readProduct(step3_output + ".dim"))
ProductIO.writeProduct(step4_product, step4_output, "BEAM-DIMAP")

CPU times: user 4.52 s, sys: 190 ms, total: 4.71 s
Wall time: 3.92 s


In [11]:
%%time
# Step: Terrain Correction
step5_output = os.path.join(FNL_OUTPUT_DIR, "corrected_output")

parameters = HashMap()
parameters.put('demName', 'SRTM 3Sec') 
parameters.put('pixelSpacingInMeter', 10.0) 
parameters.put('imgResamplingMethod', 'BILINEAR_INTERPOLATION') #BILINEAR_INTERPOLATION NEAREST_NEIGHBOUR
parameters.put('demResamplingMethod', 'BILINEAR_INTERPOLATION') 
# <saveDEM>false</saveDEM>

step5_product = GPF.createProduct("Terrain-Correction", parameters, ProductIO.readProduct(step4_output + ".dim")) 
ProductIO.writeProduct(step5_product, step5_output, 'GeoTIFF-BigTIFF')

CPU times: user 2min 49s, sys: 1.7 s, total: 2min 51s
Wall time: 2min 33s
